# Data analysis
Now that the CSV file has been cleaned, transformed and inserted into the database, we can query it and bring out some information.

Initialize database connection.

In [ ]:
from sql import db
engine = db.connect()

Import necessary libraries.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

Retrieve `monthly_restaurants_report` table data.

In [ ]:
df_source = pd.read_sql('SELECT * FROM monthly_restaurants_report', con=engine)
df_source.head()

### 📊 Number of bookings by country

In [ ]:
df = df_source.groupby('country').agg({'number_of_bookings': 'sum'}).sort_values('number_of_bookings', ascending=False)
df.plot(kind='bar', y='number_of_bookings');

💬 | UK is the country with the most bookings.

### 📊 Typiclal year
Number of bookings by month, all years combined.

In [ ]:
df = df_source.copy()
df['month'] = df.month.apply(lambda s: s.split('_')[-1])
df = df.groupby('month').agg({'number_of_bookings': 'sum'}).sort_values('month')

In [ ]:
df.plot(kind='bar', y='number_of_bookings');

💬 | December is the month with the most bookings, but the rest of the year is fairly even.

### 🇫🇷 Focus on the french restaurant with the most bookings

In [ ]:
df = df_source[df_source.country == "France"]
df = df.groupby(['restaurant_id', 'restaurant_name', 'country']).sum().reset_index()
df = df[df.number_of_bookings == df.number_of_bookings.max()]
df

💬 | The french restaurant that received the most bookings is Anovo.

Average amount by guest in Anovo.

In [ ]:
df = df_source[df_source.restaurant_id == df.restaurant_id.iloc[0]]
df['avg_amount_by_guest'] = df.amount / df.number_of_guests
df =  df[['month', 'avg_amount_by_guest']]
df['month'] = pd.to_datetime(df.month, format="%Y_%m")
df = df.set_index('month')

In [ ]:
df.mean()

💬 | This restaurant receives an average of 18.91€ per guest.

In [ ]:
df.plot(kind='line', figsize=(20,5));

💬 | We can see from the graph that three times in Anovo's history, a guest brought in more than 100€.